In [1]:
import cv2
import time
import math
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import sqlite3

# Constants
GOLDEN_RATIO = 1.618
mapped_ratio = None

# Function to remove outliers from a list
def removeOutliers(arr):
    mean = np.mean(arr)
    std = np.std(arr)
    low = mean - std * 2
    high = mean + std * 2
    return [x for x in arr if low <= x <= high]

# Function to perform face detection and calculate the golden ratio
def detectFaceAndCalculateRatio():
    global mapped_ratio
    
    top2pupil = []
    pupil2lip = []
    noseWidth = []
    nose2lips = []

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    righteye_cascade = cv2.CascadeClassifier('haarcascade_righteye.xml')
    smile_cascade = cv2.CascadeClassifier('haarcascade_mouth.xml')
    nose_cascade = cv2.CascadeClassifier('haarcascade_nose.xml')
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    startTime = time.time()

    while True:
        ret, img = cap.read()
        height, width, channels = img.shape
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + h]
            roi_color = img[y:y + h, x:x + h]
            eyes = eye_cascade.detectMultiScale(roi_gray, 2.5, 5)
            smiles = smile_cascade.detectMultiScale(roi_gray, 3.4, 5)
            noses = nose_cascade.detectMultiScale(roi_gray, 1.3, 5)
            right_eyes = righteye_cascade.detectMultiScale(roi_gray, 2.5, 5)

            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 1)
            for (sx, sy, sw, sh) in smiles:
                cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 0, 255), 1)
            for (nx, ny, nw, nh) in noses:
                cv2.rectangle(roi_color, (nx, ny), (nx + nw, ny + nh), (255, 0, 255), 1)
            if time.time() > (startTime + 2) and len(eyes) == 2 and len(smiles) == 1 and len(faces) == 1 and len(noses) == 1:
                cv2.putText(img, 'Scanning Face...', (math.floor(width / 3), math.floor(height / 12)), font, 1,
                            (0,0,0), 2, cv2.LINE_AA)
                cv2.circle(roi_color, (math.floor(w / 2), sy + math.floor(sh / 2.5)), 2, (255, 255, 255),
                           2)  # lips
                cv2.circle(roi_color, (math.floor(w / 3), ey + math.floor(eh / 2)), 2, (255, 255, 255),
                           2)  # eye
                cv2.circle(roi_color, (math.floor(nx * 1.125), ny + math.floor(nh / 2)), 2, (255, 255, 255),
                           2)  # nose R
                cv2.circle(roi_color, (nx + math.floor(nw * .875), ny + math.floor(nh / 2)), 2, (255, 255, 255),
                           2)  # nose L
                cv2.circle(roi_color, (nx + math.floor(nw / 2), ny + math.floor(nh / 2)), 2, (255, 255, 255),
                           2)  # nose
                top2pupil.append(ey + (eh / 2))
                pupil2lip.append((sy + (sh / 2.5)) - (ey + (eh / 2)))
                noseWidth.append(.75 * nw)
                nose2lips.append((sy + (sh / 3)) - (ny + (nh / 2)))

        cv2.imshow("Face Detector", img)
        k = cv2.waitKey(30) & 0xff
        if k == 27 or len(top2pupil) > 40:
            break
    cap.release()
    cv2.destroyAllWindows()
    
    top2pupil = removeOutliers(top2pupil)
    pupil2lip = removeOutliers(pupil2lip)
    noseWidth = removeOutliers(noseWidth)
    nose2lips = removeOutliers(nose2lips)
    mapped_ratio = (np.mean(top2pupil) / np.mean(pupil2lip) + np.mean(noseWidth) / np.mean(nose2lips)) / 2
    avg = (np.mean(top2pupil) / np.mean(pupil2lip) + np.mean(noseWidth) / np.mean(nose2lips)) / 2
    return img, avg  # Return the analyzed image and the normal golden ratio

# Create a function to reset the GUI
def reset_gui():
    name_entry.delete(0, tk.END)
    college_entry.delete(0, tk.END)
    student_saved_label.config(text="")
    result_label.config(text="")
    if image_label:
        image_label.config(image="")


# Function to update the GUI with the result and the analyzed image
def updateGUI(result, img):
    result_label.config(text=f"Golden Ratio: {result:.2f}")

    # Convert the OpenCV image to a PIL image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img)

    # Resize the image to fit the GUI
    pil_img = pil_img.resize((400, 400), Image.LANCZOS)

    # Convert the PIL image to a Tkinter PhotoImage
    tk_img = ImageTk.PhotoImage(pil_img)

    # Update the image in the GUI
    image_label.config(image=tk_img)
    image_label.image = tk_img

# Function to start face detection and calculate the golden ratio
def startFaceDetection():
    # Call the function to detect face and calculate the mapped ratio
    img, mapped_ratio = detectFaceAndCalculateRatio()

    # Check if the mapped_ratio is not None (valid data)
    if mapped_ratio is not None:
        # Update the GUI with the result and the analyzed image
        updateGUI(mapped_ratio, img)

        # Extract name and college values from entry fields
        name = name_entry.get()
        college = college_entry.get()

        # Check if name and college are not empty
        if name and college:
            # Save the student data to the database with the mapped golden ratio
            save_to_database(name, college, mapped_ratio)
        else:
            # Handle the case when name or college is empty
            student_saved_label.config(text="Please enter both Name and College.")
    else:
        # Handle the case when face features are not detected properly
        student_saved_label.config(text="Face features not detected properly.")

# Function to clear student data entry fields
def clear_student_data():
    name_entry.delete(0, tk.END)
    college_entry.delete(0, tk.END)
    student_saved_label.config(text="")

# Function to save data to the database and reset the GUI
def save_to_database(name, college, mapped_ratio):
    print(f"Saving data: Name='{name}', College='{college}', Golden Ratio='{mapped_ratio}'")
    
    cursor.execute("INSERT INTO students (name, college, golden_ratio) VALUES (?, ?, ?)", (name, college, mapped_ratio))
    conn.commit()
    
    print("Data saved successfully")
    
    # Display a confirmation message
    student_saved_label.config(text=f"Student '{name}' from '{college}' saved with Golden Ratio: {mapped_ratio:.2f}")
    
    # Reset the GUI
    reset_gui()

# Create a SQLite database and table (if they don't exist)
conn = sqlite3.connect("student_data.db")
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS students (
                    id INTEGER PRIMARY KEY,
                    name TEXT,
                    college TEXT,
                    golden_ratio REAL
                )''')
conn.commit()

# Create the main application window
root = tk.Tk()
root.title("Golden Ratio Face Analysis and Student Data Entry")
root.geometry("1200x800")  # Set the size of the main window

# Create a frame for the face analysis
face_analysis_frame = ttk.Frame(root)
face_analysis_frame.pack(side="left", padx=20, pady=20)

# Create a button to start face detection
start_button = ttk.Button(face_analysis_frame, text="Start Face Detection", command=startFaceDetection)
start_button.pack(pady=20)

# Create a label to display the result
result_label = ttk.Label(face_analysis_frame, text="", font=("Helvetica", 18))
result_label.pack()

# Create a label to display the analyzed image
image_label = ttk.Label(face_analysis_frame)
image_label.pack()

# Create a button to repeat the program
repeat_button = ttk.Button(face_analysis_frame, text="Repeat", command=startFaceDetection)
repeat_button.pack(pady=10)

# Create a frame for student data entry
student_data_frame = ttk.Frame(root)
student_data_frame.pack(side="right", padx=20, pady=20)

# Create a label for the student data entry section
student_data_label = ttk.Label(student_data_frame, text="Student Data Entry", font=("Helvetica", 18))
student_data_label.pack()

# Create an entry for student name
name_label = ttk.Label(student_data_frame, text="Name:")
name_label.pack()
name_entry = ttk.Entry(student_data_frame)
name_entry.pack()

# Create an entry for college name
college_label = ttk.Label(student_data_frame, text="Class & College:")
college_label.pack()
college_entry = ttk.Entry(student_data_frame)
college_entry.pack()

# Create a button to save student data
# Inside the lambda function of the "Save Student Data" button
save_button = ttk.Button(student_data_frame, text="Save Student Data", command=lambda: save_to_database(name_entry.get(), college_entry.get(), mapped_ratio))
save_button.pack()

# Create a label to display a confirmation message after saving
student_saved_label = ttk.Label(student_data_frame, text="", font=("Helvetica", 12))
student_saved_label.pack()

# Run the GUI application
root.mainloop()

# Close the database connection when the GUI is closed
conn.close()

In [5]:
import sqlite3

# Connect to the database file
conn = sqlite3.connect("student_data.db")
cursor = conn.cursor()

# Execute a SQL query to fetch data
cursor.execute("SELECT * FROM students")
data = cursor.fetchall()

# Print the data
for row in data:
    print(row)

# Close the database connection
conn.close()

(1, '', '', 1.1242821186230358)
(2, '', '', 1.0540270448360398)
(3, 'arjun', 'sju', 1.4451770822642163)
(4, 'arjun', 'sju', 1.4192342992740756)
(5, 'ar1', 'sju', 1.4106012294765304)
(6, 'ar2', 'sju', None)
(7, 'arjun2', 'sju', 1.2252088532060326)
(8, 'arjun4', 'sju', 1.5613705760493661)
(9, 'arjun5', 'sju', None)
(10, 'arjun6', 'sju', None)
(11, 'arjun', 'sju', None)
(12, 'arjun7', 'sju', None)
(13, 'arjun8', 'sju', None)
(14, 'arjun9', 'sju', None)
(15, 'arjun10', 'sju', None)
(16, 'arjun14', 'sju', 1.0561083790902381)
(17, 'arjun15', 'sju', 1.168890776492301)
(18, 'arjun16', 'sju', 1.2420791637607742)
(19, 'Gigi', 'Fashion', 1.3025093547997175)
(20, 'aswin', 'sju', 0.8958394814302233)
(21, 'Arjun', 'Sju', 1.1295688370226127)
(22, '', '', None)
(23, '', '', 1.0726073064284367)
(24, 'arjun', 'sju', 1.3797315891411246)
(25, 'arjun', 'Sju', 1.1346117484241245)
(26, '', '', None)
(27, 'try', 'worked', 1.1947191353167106)
(28, 'Umair', '', 1.132852735888767)
(29, '', '', 1.1081240783538457